# Análise Interativa de Progresso de Treinos com Hevy e Plotly

## Introdução

Este notebook tem como objetivo realizar uma análise detalhada do progresso em treinos de musculação, utilizando dados exportados do aplicativo Hevy. Através da manipulação de dados com a biblioteca Pandas e da visualização interativa com Plotly, criaremos uma página web que permite ao usuário explorar sua evolução em diferentes exercícios, rotinas e períodos de tempo.

O resultado final é uma ferramenta visual e intuitiva para acompanhar o desenvolvimento da força e performance ao longo do tempo, auxiliando na identificação de tendências, platôs e áreas de melhoria.

**Principais Ferramentas Utilizadas:**
*   **Python:** Linguagem de programação principal.
*   **Pandas:** Para manipulação e análise dos dados dos treinos.
*   **Plotly:** Para a criação de gráficos interativos e dinâmicos.
*   **HTML/CSS/JavaScript:** Para a construção da interface web que exibirá os gráficos.
*   **Jupyter Notebook:** Para o desenvolvimento e documentação do projeto.

Vamos começar configurando nosso ambiente e carregando os dados.

In [1]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import plotly.graph_objects as go
import json
from plotly.utils import PlotlyJSONEncoder

workout_df = pd.read_csv('workouts.csv')
# workout_df.head() # Para visualização opcional

## 1. Configuração Inicial e Importações

A célula de código anterior realizou as seguintes ações:
1.  **Importou as bibliotecas necessárias:**
    *   `numpy` (geralmente usado com Pandas, embora não diretamente neste script simplificado).
    *   `pandas` (`pd`): Essencial para carregar, manipular e limpar os dados do arquivo CSV exportado pelo Hevy.
    *   `datetime` e `timedelta` do módulo `datetime`: Para trabalhar com datas e calcular períodos de tempo (como "últimos 30 dias").
    *   `plotly.graph_objects` (`go`): O módulo principal do Plotly para criar figuras e traços (linhas, marcadores) nos gráficos.
    *   `json` e `PlotlyJSONEncoder` da `plotly.utils`: Para converter os objetos complexos do Plotly em um formato (JSON) que pode ser facilmente embutido e lido pelo JavaScript na página HTML.
2.  **Carregou os dados do treino:**
    *   `workout_df = pd.read_csv('workouts.csv')`: Esta linha lê o arquivo `workouts.csv` (que você deve ter no mesmo diretório do notebook ou fornecer o caminho correto) e o armazena em um DataFrame do Pandas chamado `workout_df`. Um DataFrame é como uma tabela ou planilha, otimizada para análise de dados.

Com as bibliotecas importadas e os dados brutos carregados, o próximo passo é limpar e preparar esses dados para a visualização.

In [2]:
workout_df_columns_filtered = workout_df[['title','start_time', 'exercise_title', 'set_index', 'set_type', 'weight_kg', 'reps']]
workout_df_columns_filtered = workout_df_columns_filtered.rename(columns={'start_time': 'date'})

# Converte 'date' para datetime e depois para date (sem a hora)
# Usar errors='coerce' para transformar datas inválidas em NaT, que podem ser tratadas depois se necessário
workout_df_columns_filtered['date'] = pd.to_datetime(workout_df_columns_filtered['date'], format="%d %b %Y, %H:%M", errors='coerce')
workout_df_columns_filtered = workout_df_columns_filtered.dropna(subset=['date']) # Remove linhas onde a data não pôde ser parseada
workout_df_columns_filtered['date'] = workout_df_columns_filtered['date'].dt.date

# Filtrar para os últimos 90 dias (ou o período máximo que você quer pré-processar)
hoje = datetime.now().date()
limite_maximo_historico = hoje - timedelta(days=90) # Ajuste conforme necessário

workout_df_processed = workout_df_columns_filtered[workout_df_columns_filtered['date'] >= limite_maximo_historico].copy()
# workout_df_processed.head() # Para visualização opcional

## 2. Preparação e Filtro Inicial dos Dados

Após carregar os dados brutos, é crucial prepará-los para a análise. A célula de código anterior realizou as seguintes etapas de processamento:

1.  **Seleção de Colunas Relevantes:**
    *   Filtramos o DataFrame original (`workout_df`) para manter apenas as colunas que são importantes para nossa análise de progresso:
        *   `title`: Nome da rotina de treino (ex: "Legs sábado").
        *   `start_time`: Data e hora de início do treino.
        *   `exercise_title`: Nome do exercício (ex: "Deadlift (Barbell)").
        *   `set_index`: O índice da série dentro de um exercício (0 para a primeira série, 1 para a segunda, etc.).
        *   `set_type`: Tipo da série (ex: "normal", "failure", "warmup", "drop").
        *   `weight_kg`: Peso levantado em quilogramas.
        *   `reps`: Número de repetições realizadas.
    *   Isso cria um novo DataFrame, `workout_df_columns_filtered`, mais enxuto e focado.

2.  **Renomeação da Coluna de Data:**
    *   A coluna `start_time` foi renomeada para `date` para clareza.

3.  **Conversão e Limpeza da Coluna de Data:**
    *   A coluna `date` (originalmente `start_time`) é convertida de texto para um formato de data e hora compreensível pelo Pandas (`pd.to_datetime`). Especificamos o formato esperado (`"%d %b %Y, %H:%M"`) e usamos `errors='coerce'` para que, se alguma data não puder ser convertida, ela se torne `NaT` (Not a Time) em vez de causar um erro.
    *   Linhas com datas inválidas (`NaT`) são removidas usando `dropna(subset=['date'])`.
    *   A parte da hora é removida da coluna `date`, mantendo apenas a data (`.dt.date`). Isso simplifica agrupamentos e filtros por dia.

4.  **Filtro por Período Histórico (Opcional, mas usado aqui):**
    *   Para manter o volume de dados gerenciável e focar em dados mais recentes, filtramos os treinos para incluir apenas aqueles realizados nos últimos 90 dias a partir da data atual.
    *   O resultado é armazenado em `workout_df_processed`. Esta é uma cópia do DataFrame filtrado, garantindo que modificações futuras não afetem o `workout_df_columns_filtered` original.

Com os dados limpos e pré-filtrados, podemos prosseguir para a lógica de geração dos gráficos.

In [3]:
def gerar_grafico_plotly_para_html(df, treino_nome, exercicio_nome, dias_periodo=30, serie_selecionada="Top Set (Failure)"):
    """
    Gera os dados e o layout de um gráfico Plotly para um treino, exercício, período e tipo de série específicos.
    Retorna um dicionário {'data': ..., 'layout': ...} ou None.
    """
    df_copy = df.copy()

    exercicio_df_original = df_copy[(df_copy['title'] == treino_nome) & (df_copy['exercise_title'] == exercicio_nome)]
    if exercicio_df_original.empty:
        return None

    exercicio_df = exercicio_df_original.copy()
    has_failure = not exercicio_df[exercicio_df['set_type'] == 'failure'].empty
    
    if exercicio_df['date'].empty:
        return None
    max_date = exercicio_df['date'].max()

    exercicio_df.loc[:, 'normal_order'] = pd.NA
    for dt_unique in exercicio_df['date'].unique():
        subset_indices = exercicio_df[
            (exercicio_df['date'] == dt_unique) & (exercicio_df['set_type'] == 'normal')
        ].sort_values('set_index').index
        for i, idx in enumerate(subset_indices, 1):
            exercicio_df.loc[idx, 'normal_order'] = i
    exercicio_df['normal_order'] = exercicio_df['normal_order'].astype('Int64')

    periodo_inicio = max_date - timedelta(days=dias_periodo)
    exercicio_df_periodo = exercicio_df[exercicio_df['date'] >= periodo_inicio].copy()

    if exercicio_df_periodo.empty:
        return None

    traces_data = []
    work_orders_in_period = sorted(exercicio_df_periodo[exercicio_df_periodo['normal_order'].notna()]['normal_order'].unique())

    if has_failure and (serie_selecionada == "Top Set (Failure)" or serie_selecionada == "Todas as Séries"):
        fdata = exercicio_df_periodo[exercicio_df_periodo['set_type'] == 'failure']
        if not fdata.empty:
            traces_data.append(go.Scatter(
                x=fdata['date'], y=fdata['weight_kg'],
                text=[f"{r} reps" for r in fdata['reps']],
                mode='lines+markers+text', textposition='top center',
                name='Top Set (Failure)'
            ))

    for order_val in work_orders_in_period:
        if serie_selecionada == f"Work Set {int(order_val)}" or serie_selecionada == "Todas as Séries":
            wdata = exercicio_df_periodo[exercicio_df_periodo['normal_order'] == order_val]
            if not wdata.empty:
                traces_data.append(go.Scatter(
                    x=wdata['date'], y=wdata['weight_kg'],
                    text=[f"{r} reps" for r in wdata['reps']],
                    mode='lines+markers+text', textposition='top center',
                    name=f'Work Set {int(order_val)}'
                ))
    
    if not traces_data:
        return None

    layout_title = f"Evolução: {exercicio_nome} ({treino_nome})"
    if serie_selecionada != "Todas as Séries":
        layout_title += f" - {serie_selecionada}"
    layout_title += f" - Últimos {dias_periodo} dias"

    layout_dict = go.Layout(
        title=layout_title,
        xaxis=dict(title='Data', tickformat='%d/%m/%Y'),
        yaxis=dict(title='Peso (kg)'),
        hovermode='closest',
        margin=dict(t=80, b=50, l=50, r=30) 
    ).to_plotly_json()

    return {'data': [trace.to_plotly_json() for trace in traces_data], 'layout': layout_dict}

## 3. Função para Geração das Especificações do Gráfico

A célula anterior define a função `gerar_grafico_plotly_para_html`. Esta função é o coração da visualização de cada exercício individual e é projetada para ser chamada pela lógica que prepara todos os dados para a página HTML.

**Principais Responsabilidades da Função:**

1.  **Receber Parâmetros Específicos:**
    *   `df`: O DataFrame processado contendo todos os dados de treino.
    *   `treino_nome`: O nome da rotina de treino a ser filtrada.
    *   `exercicio_nome`: O nome do exercício específico para o qual o gráfico será gerado.
    *   `dias_periodo`: Um número inteiro (ex: 30, 60, 90) definindo o período de tempo (em dias, retrocedendo da data mais recente do exercício) para incluir no gráfico. O padrão é 30 dias.
    *   `serie_selecionada`: Uma string que indica qual(is) tipo(s) de série(s) devem ser exibidas. Exemplos:
        *   `"Top Set (Failure)"`: Mostra apenas as séries marcadas como "failure" (geralmente a última e mais pesada série de um exercício).
        *   `"Work Set 1"`: Mostra apenas a primeira série de trabalho (não "failure").
        *   `"Todas as Séries"`: Mostra o "Top Set (Failure)" (se existir) e todas as séries de trabalho ("normal").

2.  **Filtragem Adicional dos Dados:**
    *   Filtra o DataFrame fornecido para o `treino_nome` e `exercicio_nome` especificados.
    *   Identifica se existem séries do tipo "failure" (`has_failure`).
    *   Determina a data mais recente (`max_date`) para o exercício e calcula a data de início do período com base em `dias_periodo`.
    *   Filtra novamente os dados para incluir apenas os registros dentro do período selecionado.

3.  **Ordenação das Séries de Trabalho ("Normal Order"):**
    *   Para diferenciar múltiplas séries de trabalho ("normal") realizadas no mesmo dia para o mesmo exercício, uma nova coluna `normal_order` é criada. Ela numera sequencialmente (1, 2, 3...) as séries "normais" com base em seu `set_index` original para cada dia.

4.  **Criação das Traces (Linhas do Gráfico):**
    *   Itera sobre os tipos de séries a serem exibidas com base no parâmetro `serie_selecionada`.
    *   Para cada série a ser mostrada, cria um objeto `go.Scatter` do Plotly. Este objeto define:
        *   `x`: As datas dos treinos.
        *   `y`: Os pesos (`weight_kg`) levantados.
        *   `text`: Informações adicionais para exibição ao passar o mouse (hover), como o número de repetições.
        *   `mode`: Como os dados são exibidos (linhas, marcadores e texto sobre os pontos).
        *   `name`: O nome da série que aparecerá na legenda do gráfico (ex: "Top Set (Failure)", "Work Set 1").
    *   Apenas as traces com dados dentro do período e correspondentes à `serie_selecionada` são criadas.

5.  **Definição do Layout do Gráfico:**
    *   Cria um objeto `go.Layout` que define a aparência geral do gráfico:
        *   `title`: Um título dinâmico que inclui o nome do exercício, rotina, tipo de série e período.
        *   `xaxis`, `yaxis`: Rótulos e formatação para os eixos X (Data) e Y (Peso).
        *   `hovermode`: Comportamento do hover.
        *   `margin`: Ajustes nas margens para melhor visualização do título.

6.  **Retorno da Especificação do Gráfico:**
    *   A função não retorna um objeto de figura Plotly interativo diretamente, mas sim um **dicionário** contendo duas chaves:
        *   `'data'`: Uma lista das especificações de cada trace (convertidas para formato JSON).
        *   `'layout'`: A especificação do layout (convertida para formato JSON).
    *   Este formato é ideal para ser serializado em JSON e depois usado pelo JavaScript na página HTML para renderizar o gráfico com `Plotly.newPlot()`.

Se não houver dados para a combinação especificada, a função retorna `None`. Esta função modular permite gerar a "receita" para qualquer gráfico sob demanda.

In [4]:
def gerar_dados_para_js(df, rotinas_exercicios_map):
    graficos_data_para_js = {}
    periodos_dias_options = [30, 60, 90]
    
    for rotina, exercicios in rotinas_exercicios_map.items():
        for exercicio in exercicios:
            temp_exercicio_df = df[(df['title'] == rotina) & (df['exercise_title'] == exercicio)]
            if temp_exercicio_df.empty or temp_exercicio_df['date'].empty:
                continue

            series_disponiveis_para_este_exercicio = ["Todas as Séries"]
            if not temp_exercicio_df[temp_exercicio_df['set_type'] == 'failure'].empty:
                series_disponiveis_para_este_exercicio.append("Top Set (Failure)")
            
            # Certifique-se de que max_date_temp é calculado sobre um DataFrame não vazio
            max_date_temp = temp_exercicio_df['date'].max() 
            ninety_days_ago_temp = max_date_temp - timedelta(days=90)
            
            temp_exercicio_df_copy = temp_exercicio_df.copy()
            temp_exercicio_df_copy.loc[:, 'normal_order_temp'] = pd.NA
            for dt_unique_temp in temp_exercicio_df_copy['date'].unique():
                subset_indices_temp = temp_exercicio_df_copy[
                    (temp_exercicio_df_copy['date'] == dt_unique_temp) & 
                    (temp_exercicio_df_copy['set_type'] == 'normal')
                ].sort_values('set_index').index
                for i_temp, idx_temp in enumerate(subset_indices_temp, 1):
                    temp_exercicio_df_copy.loc[idx_temp, 'normal_order_temp'] = i_temp
            temp_exercicio_df_copy['normal_order_temp'] = temp_exercicio_df_copy['normal_order_temp'].astype('Int64')

            potential_work_orders = sorted(temp_exercicio_df_copy[
                (temp_exercicio_df_copy['normal_order_temp'].notna()) &
                (temp_exercicio_df_copy['date'] >= ninety_days_ago_temp) 
            ]['normal_order_temp'].unique())
            
            for ws_order in potential_work_orders:
                series_disponiveis_para_este_exercicio.append(f"Work Set {int(ws_order)}")

            for periodo_val in periodos_dias_options:
                for tipo_serie_val in series_disponiveis_para_este_exercicio:
                    chave_js = f"{rotina}||{exercicio}||{periodo_val}||{tipo_serie_val}"
                    grafico_spec = gerar_grafico_plotly_para_html(df, rotina, exercicio, 
                                                                  dias_periodo=periodo_val, 
                                                                  serie_selecionada=tipo_serie_val)
                    if grafico_spec:
                        graficos_data_para_js[chave_js] = grafico_spec
                        
    return graficos_data_para_js

## 4. Preparação de Todas as Combinações de Gráficos para a Interface

A célula anterior define a função `gerar_dados_para_js`. O propósito desta função é iterar sobre todas as rotinas, exercícios, períodos de tempo desejados e tipos de séries relevantes, e para cada combinação válida, gerar a especificação do gráfico usando a função `gerar_grafico_plotly_para_html` que definimos anteriormente.

**Como Funciona:**

1.  **Recebe Dados e Mapa de Rotinas:**
    *   `df`: O DataFrame `workout_df_processed` com os dados de treino já filtrados (ex: últimos 90 dias).
    *   `rotinas_exercicios_map`: Um dicionário Python onde as chaves são nomes de rotinas e os valores são listas de nomes de exercícios pertencentes a cada rotina.

2.  **Iteração Abrangente:**
    *   A função percorre cada `rotina` e cada `exercicio` dentro dessa rotina.
    *   Para cada exercício, ela define uma lista de `periodos_dias_options` (ex: 30, 60, 90 dias).
    *   **Determinação Dinâmica dos Tipos de Séries:**
        *   Para um dado exercício, ela verifica quais tipos de séries são realmente aplicáveis (ex: se existe "Top Set (Failure)", quais "Work Sets" como "Work Set 1", "Work Set 2" etc., foram registrados nos últimos 90 dias para nomeação consistente). A opção "Todas as Séries" é sempre considerada.
        *   Isso evita gerar combinações para tipos de séries que não existem para um exercício específico, tornando a interface mais limpa.

3.  **Geração da Chave e Especificação do Gráfico:**
    *   Para cada combinação válida de `rotina`, `exercicio`, `periodo_val` e `tipo_serie_val`, uma **chave única** no formato string é criada: `"rotina||exercicio||periodo||serie_tipo"`. Esta chave será usada no JavaScript para encontrar a especificação correta do gráfico.
    *   A função `gerar_grafico_plotly_para_html` é chamada com os parâmetros da combinação atual.

4.  **Armazenamento das Especificações:**
    *   Se `gerar_grafico_plotly_para_html` retornar uma especificação válida (ou seja, não `None`), essa especificação (o dicionário `{'data': ..., 'layout': ...}`) é armazenada no dicionário principal `graficos_data_para_js`, usando a chave única gerada.

5.  **Retorno do Dicionário Completo:**
    *   Ao final de todas as iterações, a função retorna `graficos_data_para_js`. Este dicionário contém todas as "receitas" de gráficos pré-calculadas, prontas para serem convertidas em JSON e usadas pela página HTML.

Essa abordagem de pré-gerar todas as especificações garante que, quando o usuário fizer uma seleção na interface HTML, os dados e o layout do gráfico já estejam prontos, resultando em um carregamento rápido e eficiente do gráfico desejado.

In [5]:
# Criar mapa de rotinas para exercícios para o JavaScript
rotinas_exercicios_js_map = {}
treino_nomes_unicos = workout_df_processed['title'].unique()
for nome_treino_map in treino_nomes_unicos:
    exercicios_map_lista = workout_df_processed[workout_df_processed['title'] == nome_treino_map]['exercise_title'].unique().tolist()
    rotinas_exercicios_js_map[nome_treino_map] = sorted(exercicios_map_lista) # Ordenar para consistência

# Gerar o dicionário de especificações de gráficos para JS
graficos_plotly_specs_para_js = gerar_dados_para_js(workout_df_processed, rotinas_exercicios_js_map)

# Serializar para JSON
js_plotly_data_var = f"const graficosPlotlySpecs = {json.dumps(graficos_plotly_specs_para_js, cls=PlotlyJSONEncoder)};"
js_rotinas_map_var = f"const rotinasExerciciosMap = {json.dumps(rotinas_exercicios_js_map)};"

print(f"Número de especificações de gráfico geradas: {len(graficos_plotly_specs_para_js)}")
if graficos_plotly_specs_para_js:
   print(f"Exemplo de chave JS: {list(graficos_plotly_specs_para_js.keys())[0]}")

Número de especificações de gráfico geradas: 305
Exemplo de chave JS: Legs sábado||Deadlift (Barbell)||30||Todas as Séries


## 5. Serialização dos Dados para Uso no HTML/JavaScript

Com todas as especificações de gráficos preparadas pela função `gerar_dados_para_js`, a próxima etapa crucial é converter essa estrutura de dados Python em um formato que possa ser facilmente embutido e utilizado pelo JavaScript na nossa página HTML. Para isso, usamos o formato JSON (JavaScript Object Notation).

A célula de código anterior executa as seguintes tarefas:

1.  **Criação do Mapa de Rotinas e Exercícios para os Seletores HTML (`rotinas_exercicios_js_map`):**
    *   Primeiro, ela cria um dicionário Python chamado `rotinas_exercicios_js_map`.
    *   Este dicionário é estruturado para facilitar a criação dos menus suspensos (dropdowns) na página HTML. As chaves são os nomes das rotinas de treino, e os valores são listas (ordenadas) dos nomes de exercícios únicos pertencentes a cada rotina.
    *   Isso permite que, quando o usuário selecionar uma rotina no HTML, o dropdown de exercícios seja populado dinamicamente apenas com os exercícios relevantes.

2.  **Geração do Dicionário Principal de Especificações de Gráficos:**
    *   A função `gerar_dados_para_js` (definida na célula anterior) é chamada, passando o DataFrame `workout_df_processed` e o `rotinas_exercicios_js_map`.
    *   O resultado, `graficos_plotly_specs_para_js`, é um grande dicionário onde cada chave representa uma combinação única de rotina, exercício, período e tipo de série, e o valor é a especificação Plotly (`{'data': ..., 'layout': ...}`) para essa combinação.

3.  **Conversão para Strings JavaScript (JSON):**
    *   `js_plotly_data_var = f"const graficosPlotlySpecs = {json.dumps(graficos_plotly_specs_para_js, cls=PlotlyJSONEncoder)};"`:
        *   `json.dumps()` converte o dicionário Python `graficos_plotly_specs_para_js` em uma string formatada como JSON.
        *   `cls=PlotlyJSONEncoder` é importante porque os objetos Plotly (como datas ou arrays NumPy que podem estar dentro das especificações) precisam de um codificador especial para serem convertidos corretamente para JSON.
        *   O resultado é uma string que define uma constante JavaScript `graficosPlotlySpecs`, contendo todo o nosso catálogo de gráficos.
    *   `js_rotinas_map_var = f"const rotinasExerciciosMap = {json.dumps(rotinas_exercicios_js_map)};"`:
        *   Da mesma forma, o dicionário `rotinas_exercicios_js_map` é convertido em uma string JSON e atribuído a uma constante JavaScript `rotinasExerciciosMap`.

4.  **Feedback (Opcional):**
    *   As linhas `print()` no final fornecem um feedback sobre quantas especificações de gráficos foram geradas e um exemplo de chave, o que é útil para depuração e para confirmar que o processo ocorreu como esperado.

Com essas duas strings JavaScript (`js_plotly_data_var` e `js_rotinas_map_var`), temos tudo o que precisamos para injetar os dados e a estrutura de navegação na nossa página HTML.

In [ ]:
html_template_final = f"""
<!DOCTYPE html>
<html lang="pt-BR">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Análise de Progresso de Treinos - Hevy</title>
    <script src="https://cdn.plot.ly/plotly-latest.min.js"></script>
    <link href="https://fonts.googleapis.com/css2?family=Roboto:wght@300;400;500;700&display=swap" rel="stylesheet">
    <style>
        :root {{
            --primary-color: #007bff; /* Azul primário */
            --primary-hover-color: #0056b3;
            --secondary-color: #6c757d; /* Cinza secundário */
            --light-gray-color: #f8f9fa;
            --medium-gray-color: #e9ecef;
            --dark-gray-color: #343a40;
            --success-color: #28a745;
            --danger-color: #dc3545;
            --warning-color: #ffc107;
            --info-bg-color: #e2f3ff;
            --info-border-color: #b8daff;
            --info-text-color: #0c5464;
            --border-radius: 0.375rem; /* 6px */
            --box-shadow: 0 0.5rem 1rem rgba(0, 0, 0, 0.1);
            --input-focus-shadow: 0 0 0 0.25rem rgba(0, 123, 255, 0.25);
        }}

        * {{
            box-sizing: border-box;
            margin: 0;
            padding: 0;
        }}

        body {{ 
            font-family: 'Roboto', 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif; 
            background-color: var(--light-gray-color); 
            color: var(--dark-gray-color); 
            line-height: 1.6; 
            font-size: 16px;
        }}

        header {{ 
            background-color: var(--primary-color); 
            color: white; 
            padding: 20px 25px; 
            text-align: center; 
            box-shadow: 0 2px 5px rgba(0,0,0,0.15); 
            margin-bottom: 30px;
        }}
        header h1 {{ 
            margin: 0; 
            font-size: 2em; 
            font-weight: 500;
        }}

        .container {{ 
            max-width: 1140px; /* Aumentado para layouts mais amplos */
            margin: 0 auto 30px auto; 
            padding: 30px; 
            background-color: #ffffff; 
            box-shadow: var(--box-shadow); 
            border-radius: var(--border-radius); 
        }}

        .controls-grid {{ 
            display: grid; 
            grid-template-columns: repeat(auto-fit, minmax(240px, 1fr)); /* Ajustado minmax */
            gap: 20px; 
            margin-bottom: 25px; 
        }}
        .control-group label {{ 
            font-weight: 500; 
            display: block; 
            margin-bottom: 8px; 
            font-size: 0.95em; 
            color: #495057; 
        }}
        select {{ 
            width: 100%; 
            padding: 12px 15px; 
            font-size: 1em; 
            border: 1px solid #ced4da; 
            border-radius: var(--border-radius); 
            background-color: #fff; 
            transition: border-color 0.2s ease-in-out, box-shadow 0.2s ease-in-out; 
            appearance: none; /* Remove a seta padrão do browser */
            background-image: url('data:image/svg+xml;charset=US-ASCII,%3Csvg%20width%3D%2220%22%20height%3D%2220%22%20xmlns%3D%22http%3A//www.w3.org/2000/svg%22%3E%3Cpath%20d%3D%22M5%208l5%205%205-5z%22%20fill%3D%22%23555%22/%3E%3C/svg%3E');
            background-repeat: no-repeat;
            background-position: right 15px center;
            background-size: 12px;
        }}
        select:focus {{ 
            border-color: var(--primary-color); 
            outline: 0; 
            box-shadow: var(--input-focus-shadow);
        }}

        #grafico-container {{ 
            width: 100%; 
            min-height: 500px; /* Aumentado */
            height: 520px; 
            margin-top: 25px; 
            border: 1px solid var(--medium-gray-color); 
            border-radius: var(--border-radius); 
            background-color: #fff; 
            display: block;
            position: relative;
            overflow: hidden; 
        }}
        #grafico-container p {{ 
            text-align: center;  
            color: var(--secondary-color); 
            font-size: 1.1em; 
            position: absolute;
            top: 50%;
            left: 50%;
            transform: translate(-50%, -50%);
            width: 90%;
        }}
        
        .loader-wrapper {{
            display: flex;
            flex-direction: column;
            align-items: center;
            justify-content: center;
            position: absolute;
            top: 0;
            left: 0;
            width: 100%;
            height: 100%;
            background-color: rgba(255, 255, 255, 0.85); 
            z-index: 10; 
        }}
        .loader {{
            border: 6px solid #e9ecef; /* Cinza mais claro */
            border-top: 6px solid var(--primary-color); /* Azul primário */
            border-radius: 50%;
            width: 50px;
            height: 50px;
            animation: spin 0.8s linear infinite;
        }}
        @keyframes spin {{
            0% {{ transform: rotate(0deg); }}
            100% {{ transform: rotate(360deg); }}
        }}

        .alert {{ 
            background-color: var(--info-bg-color); 
            color: var(--info-text-color); 
            border: 1px solid var(--info-border-color); 
            padding: 18px; 
            border-radius: var(--border-radius); 
            margin-bottom: 30px; 
            font-size: 0.95em; 
        }}
        .alert strong {{ 
            color: #004085; /* Um pouco mais escuro para destaque */
            font-weight: 500;
        }}
        .alert ul {{
            margin-top: 10px;
            margin-left: 20px;
            padding-left: 0;
        }}
        .alert li {{
            margin-bottom: 5px;
        }}

        @media (max-width: 768px) {{
            .controls-grid {{ grid-template-columns: 1fr; }}
            header h1 {{ font-size: 1.6em; }}
            .container {{ margin: 15px; padding: 15px; }}
            select {{ padding: 10px 12px; background-position: right 12px center;}}
            #grafico-container {{ min-height: 400px; height: 420px; }}
        }}
    </style>
</head>
<body>
    <header><h1>Análise de Progresso de Treinos</h1></header>
    <div class="container">
        <div class="alert">
            <strong>Bem-vindo(a) à sua Análise de Progresso de Treinos!</strong>
            <ul>
                <li>Comece selecionando uma <strong>Rotina de Treino</strong> e, em seguida, um <strong>Exercício</strong> específico.</li>
                <li>O gráfico será carregado automaticamente com os dados dos últimos 30 dias para o "Top Set" (a série mais pesada até a falha) ou a primeira série de trabalho disponível.</li>
                <li>Utilize os seletores de <strong>Período</strong> e <strong>Tipo de Série</strong> para explorar diferentes visualizações do seu progresso. O gráfico se atualizará automaticamente.</li>
                <li>Passe o mouse (ou clique no caso de celulares) sobre os <strong>pontos do gráfico</strong> para ver detalhes como data, peso e repetições.</li>
                <li><strong>Dica:</strong> Para uma melhor visualização em celulares, experimente usar o aparelho na horizontal (modo paisagem) ou ativar a opção "Versão para computador" no seu navegador.</li>
            </ul>
        </div>

        <div class="controls-grid">
            <div class="control-group">
                <label for="rotina">Rotina de Treino:</label>
                <select id="rotina" onchange="rotinaSelecionada()">
                    <option value="">-- Selecione uma Rotina --</option>
                </select>
            </div>
            <div class="control-group">
                <label for="exercicio">Exercício:</label>
                <select id="exercicio" onchange="exercicioSelecionado()">
                    <option value="">-- Selecione um Exercício --</option>
                </select>
            </div>
            <div class="control-group">
                <label for="periodo">Período de Análise:</label>
                <select id="periodo" onchange="periodoOuSerieTipoAlterado(event)">
                    <option value="30" selected>Últimos 30 dias</option>
                    <option value="60">Últimos 60 dias</option>
                    <option value="90">Últimos 90 dias</option>
                </select>
            </div>
            <div class="control-group">
                <label for="serie_tipo">Visualizar Série(s):</label>
                <select id="serie_tipo" onchange="periodoOuSerieTipoAlterado(event)">
                    <option value="">-- Selecione o Tipo de Série --</option>
                </select>
            </div>
        </div>
        <div id="grafico-container"><p>Selecione uma rotina e um exercício para iniciar a análise.</p></div>
    </div>

    <script>
        {js_plotly_data_var}
        {js_rotinas_map_var}

        const rotinaSelect = document.getElementById('rotina');
        const exercicioSelect = document.getElementById('exercicio');
        const periodoSelect = document.getElementById('periodo');
        const serieTipoSelect = document.getElementById('serie_tipo');
        const graficoContainer = document.getElementById('grafico-container');

        function popularRotinas() {{
            for (const rotinaNome in rotinasExerciciosMap) {{
                if (rotinasExerciciosMap.hasOwnProperty(rotinaNome)) {{ 
                    const option = document.createElement('option');
                    option.value = rotinaNome;
                    option.text = rotinaNome;
                    rotinaSelect.appendChild(option);
                }}
            }}
        }}

        function rotinaSelecionada() {{
            const rotinaNome = rotinaSelect.value;
            exercicioSelect.innerHTML = '<option value=\"\">-- Selecione um Exercício --</option>'; 
            serieTipoSelect.innerHTML = '<option value=\"\">-- Selecione o Tipo de Série --</option>'; 
            graficoContainer.innerHTML = '<p>Selecione uma rotina e um exercício para iniciar a análise.</p>';

            if (rotinaNome && rotinasExerciciosMap[rotinaNome]) {{
                rotinasExerciciosMap[rotinaNome].forEach(exercicioNome => {{
                    const option = document.createElement('option');
                    option.value = exercicioNome;
                    option.text = exercicioNome;
                    exercicioSelect.appendChild(option);
                }});
            }}
             // Não chama exercicioSelecionado() aqui, espera a ação do usuário no dropdown de exercício
        }}

        function exercicioSelecionado() {{
            const rotinaNome = rotinaSelect.value;
            const exercicioNome = exercicioSelect.value;
            
            const currentSerieTipo = serieTipoSelect.value; 
            serieTipoSelect.innerHTML = '<option value=\"\">-- Selecione o Tipo de Série --</option>';
            
            if (!rotinaNome || !exercicioNome) {{ // Se o exercício for desmarcado
                graficoContainer.innerHTML = '<p>Selecione uma rotina e um exercício para iniciar a análise.</p>';
                return;
            }}

            const seriesSet = new Set();
            const periodoAtualParaPopularSeries = periodoSelect.value || '30'; 

            for (const chave in graficosPlotlySpecs) {{
                if (graficosPlotlySpecs.hasOwnProperty(chave)) {{
                    const partes = chave.split('||');
                    if (partes[0] === rotinaNome && partes[1] === exercicioNome && partes[2] === periodoAtualParaPopularSeries) {{
                        seriesSet.add(partes[3]);
                    }}
                }}
            }}
            
            if (seriesSet.size === 0 && periodoAtualParaPopularSeries !== '30') {{
                 for (const chave in graficosPlotlySpecs) {{
                     if (graficosPlotlySpecs.hasOwnProperty(chave)) {{
                        const partes = chave.split('||');
                        if (partes[0] === rotinaNome && partes[1] === exercicioNome && partes[2] === '30') {{
                            seriesSet.add(partes[3]);
                        }}
                    }}
                }}
            }}
            if (seriesSet.size === 0) {{ 
                for (const p of ['30', '60', '90']) {{
                    for (const chave in graficosPlotlySpecs) {{
                        if (graficosPlotlySpecs.hasOwnProperty(chave)) {{
                            const partes = chave.split('||');
                            if (partes[0] === rotinaNome && partes[1] === exercicioNome && partes[2] === p) {{
                                seriesSet.add(partes[3]);
                            }}
                        }}
                    }}
                    if (seriesSet.size > 0) break;
                }}
            }}

            const seriesOrder = ["Top Set (Failure)", "Todas as Séries"];
            const sortedSeries = Array.from(seriesSet).sort((a, b) => {{
                let aIdx = seriesOrder.indexOf(a);
                let bIdx = seriesOrder.indexOf(b);
                if (a.startsWith("Work Set")) aIdx = seriesOrder.length + parseInt(a.substring(9), 10); 
                if (b.startsWith("Work Set")) bIdx = seriesOrder.length + parseInt(b.substring(9), 10);
                aIdx = (aIdx === -1 || isNaN(aIdx)) ? Infinity : aIdx;
                bIdx = (bIdx === -1 || isNaN(bIdx)) ? Infinity : bIdx;
                return aIdx - bIdx;
            }});

            let restaurouSerie = false;
            if (sortedSeries.length > 0) {{
                sortedSeries.forEach(serieNome => {{
                    const option = document.createElement('option');
                    option.value = serieNome;
                    option.text = serieNome;
                    serieTipoSelect.appendChild(option);
                    if (serieNome === currentSerieTipo) {{
                        serieTipoSelect.value = currentSerieTipo;
                        restaurouSerie = true;
                    }}
                }});
            
                if (!restaurouSerie) {{ // Se a série anterior não existe mais para este período/exercício
                    if (seriesSet.has("Top Set (Failure)")) {{
                        serieTipoSelect.value = "Top Set (Failure)";
                    }} else if (seriesSet.has("Todas as Séries")) {{
                        serieTipoSelect.value = "Todas as Séries";
                    }} else {{
                        serieTipoSelect.value = sortedSeries[0]; // Primeira disponível da lista ordenada
                    }}
                }}
            }} else {{
                serieTipoSelect.innerHTML = '<option value=\"\">Nenhuma série aplicável</option>';
            }}
            
            carregarGrafico(); 
        }}
        
        function periodoOuSerieTipoAlterado(event) {{ 
            if (event && event.target && event.target.id === 'periodo') {{
                 exercicioSelecionado(); // Repopula séries e recarrega o gráfico
                 return; 
            }}
            // Se apenas o tipo de série mudou, e já temos rotina/exercício, recarrega o gráfico.
            const rotina = rotinaSelect.value;
            const exercicio = exercicioSelect.value;
            if (rotina && exercicio) {{ 
                carregarGrafico();
            }}
        }}

        function carregarGrafico() {{
            const rotina = rotinaSelect.value;
            const exercicio = exercicioSelect.value;
            const periodo = periodoSelect.value;
            const serieTipo = serieTipoSelect.value;

            graficoContainer.innerHTML = ''; 

            if (!rotina || !exercicio || !periodo || !serieTipo || serieTipo === "" || serieTipoSelect.options[0].text === "Nenhuma série aplicável") {{
                let msg = "<p>";
                if (!rotina || !exercicio) {{
                     msg += "Selecione uma rotina e um exercício.";
                }} else if (!serieTipo || serieTipo === "" || (serieTipoSelect.options.length > 0 && serieTipoSelect.options[0].text === "Nenhuma série aplicável" && serieTipoSelect.value === "")) {{ 
                     msg += "Nenhum tipo de série aplicável ou selecione um tipo de série válido.";
                }} else {{ 
                     msg += "Por favor, selecione todas as opções válidas.";
                }}
                msg += "</p>";
                graficoContainer.innerHTML = msg;
                return;
            }}

            const chave = `${{rotina}}||${{exercicio}}||${{periodo}}||${{serieTipo}}`;
            
            const loaderWrapper = document.createElement('div');
            loaderWrapper.className = 'loader-wrapper';
            const loaderDiv = document.createElement('div');
            loaderDiv.className = 'loader';
            const loadingText = document.createElement('p');
            loadingText.textContent = 'Carregando gráfico...';
            
            loaderWrapper.appendChild(loaderDiv);
            loaderWrapper.appendChild(loadingText);
            graficoContainer.appendChild(loaderWrapper);

            setTimeout(() => {{ 
                graficoContainer.innerHTML = ''; 
                if (graficosPlotlySpecs && graficosPlotlySpecs[chave]) {{
                    const graficoSpec = graficosPlotlySpecs[chave];
                    Plotly.newPlot(graficoContainer, graficoSpec.data, graficoSpec.layout, {{responsive: true, displaylogo: false}});
                }} else {{
                    graficoContainer.innerHTML = "<p>Dados não disponíveis para esta combinação. <br>Verifique se há registros para o período e tipo de série selecionados ou tente outra combinação.</p>";
                }}
            }}, 50);
        }}

        document.addEventListener('DOMContentLoaded', () => {{
            popularRotinas();
        }});
    </script>
</body>
</html>
"""

with open("index.html", "w", encoding="utf-8") as f:
    f.write(html_template_final)

print("Arquivo HTML 'index.html' gerado.")

Arquivo HTML 'visualizacao_treino_profissional.html' gerado.


## 6. Construção da Página Web Interativa e Geração do Arquivo HTML

A célula final do nosso notebook é responsável por construir a interface do usuário (UI) em HTML, incorporar a lógica de interatividade com JavaScript e, finalmente, salvar tudo como um arquivo `.html` que pode ser aberto em qualquer navegador web.

**Componentes da Célula:**

1.  **Template HTML (`html_template_final`):**
    *   Um f-string Python multilinha é usado para definir toda a estrutura da página HTML.
    *   **`<head>`:**
        *   Configurações básicas como `charset` e `viewport`.
        *   Um título descritivo para a página.
        *   Inclusão da biblioteca Plotly.js via CDN, essencial para renderizar os gráficos.
        *   Inclusão de uma fonte customizada (Roboto) do Google Fonts para uma estética mais moderna.
        *   Uma seção `<style>` contendo todo o CSS para estilizar a página, tornando-a visualmente agradável e profissional. O CSS define:
            *   Cores, fontes e layout geral.
            *   Estilos para o cabeçalho, contêiner principal, e a grade de controles.
            *   Aparência customizada para os elementos `<select>` (menus suspensos).
            *   Estilos para a área do gráfico (`#grafico-container`) e para o indicador de carregamento (spinner).
            *   Um bloco de alerta (`.alert`) para fornecer instruções claras ao usuário.
            *   Regras de responsividade (`@media`) para adaptar o layout a telas menores (como as de celulares).
    *   **`<body>`:**
        *   **`<header>`:** Um cabeçalho simples com o título da aplicação.
        *   **`<div class="container">`:** O contêiner principal que centraliza o conteúdo.
        *   **`<div class="alert">`:** Exibe as instruções de uso de forma clara e didática.
        *   **`<div class="controls-grid">`:** Organiza os seletores (dropdowns) em uma grade responsiva.
            *   Quatro seletores são definidos: "Rotina de Treino", "Exercício", "Período de Análise" e "Visualizar Série(s)".
            *   Cada seletor tem um `id` para referência no JavaScript e um evento `onchange` para acionar funções JavaScript quando seu valor é alterado.
        *   **`<div id="grafico-container">`:** A área designada onde o gráfico Plotly será renderizado. Inicialmente, exibe uma mensagem instruindo o usuário.
        *   **`<script>`:** Esta é a seção mais crucial para a interatividade.
            *   **Injeção de Dados:** As variáveis Python `js_plotly_data_var` (contendo `graficosPlotlySpecs`) e `js_rotinas_map_var` (contendo `rotinasExerciciosMap`) são injetadas aqui. Isso disponibiliza todas as especificações de gráficos e o mapa de rotinas/exercícios para o JavaScript.
            *   **Referências a Elementos DOM:** Constantes são criadas para referenciar os seletores e o contêiner do gráfico, facilitando a manipulação.
            *   **Função `popularRotinas()`:** Preenche o dropdown de rotinas com base no `rotinasExerciciosMap`.
            *   **Função `rotinaSelecionada()`:** É chamada quando uma rotina é selecionada. Ela limpa e repopula o dropdown de exercícios com os exercícios correspondentes à rotina escolhida. Em seguida, chama `exercicioSelecionado()`.
            *   **Função `exercicioSelecionado()`:** É chamada quando um exercício é selecionado (ou quando `rotinaSelecionada()` a invoca). Ela limpa e repopula dinamicamente o dropdown "Tipo de Série" com as opções relevantes para a combinação atual de rotina, exercício e período (buscando as séries disponíveis em `graficosPlotlySpecs`). Também define uma série padrão (Top Set, Todas, ou a primeira da lista) e, crucialmente, chama `carregarGrafico()` para exibir o gráfico inicial para essa seleção.
            *   **Função `periodoOuSerieTipoAlterado(event)`:** Acionada pela mudança nos seletores de período ou tipo de série. Se o período mudou, ela chama `exercicioSelecionado()` (que por sua vez recarrega o gráfico). Se apenas o tipo de série mudou, ela chama `carregarGrafico()` diretamente, garantindo que o gráfico seja atualizado automaticamente.
            *   **Função `carregarGrafico()`:** Esta é a função principal de renderização. Ela coleta os valores selecionados nos quatro dropdowns, constrói a chave correspondente, limpa o contêiner do gráfico, exibe um indicador de carregamento (spinner e texto), e então usa `Plotly.newPlot()` para desenhar o gráfico usando a especificação encontrada em `graficosPlotlySpecs`. Se nenhuma especificação for encontrada, exibe uma mensagem de erro. Um pequeno `setTimeout` é usado para garantir que o feedback de carregamento seja visível antes da renderização do Plotly.
            *   **Inicialização (`DOMContentLoaded`):** Quando a página HTML está totalmente carregada, `popularRotinas()` é chamada para preencher o primeiro dropdown.

2.  **Escrita do Arquivo HTML:**
    *   O conteúdo do `html_template_final` (que agora é uma string HTML completa e bem formada) é escrito em um arquivo chamado `index.html` no mesmo diretório do notebook.
    *   Uma mensagem de confirmação é impressa.

Ao abrir o arquivo `index.html` em um navegador, o usuário terá uma interface interativa e profissional para explorar seus dados de treino.

## Conclusão e Próximos Passos

Este notebook demonstrou com sucesso como transformar dados brutos de treino do aplicativo Hevy em uma visualização web interativa e profissional. Utilizamos Pandas para processamento de dados, Plotly para geração de gráficos dinâmicos, e HTML/CSS/JavaScript para criar uma interface de usuário intuitiva.

**Principais Funcionalidades Implementadas:**

*   Carregamento e limpeza de dados de treino.
*   Filtragem de dados por rotina, exercício, período de tempo e tipo de série.
*   Geração dinâmica de gráficos de progresso (peso vs. data) com indicação de repetições.
*   Interface web amigável com seletores para navegação e atualização automática dos gráficos.
*   Design responsivo e visualmente agradável.

**Possíveis Próximos Passos e Melhorias:**

1.  **Mais Tipos de Gráficos:**
    *   Gráficos de volume total (peso x reps x séries).
    *   Gráficos de PR (Recordes Pessoais) ao longo do tempo.
    *   Comparação de diferentes exercícios.
2.  **Métricas Adicionais:**
    *   Cálculo de 1RM estimado.
    *   Taxa de progressão.
3.  **Persistência de Seleções:**
    *   Usar `localStorage` no navegador para lembrar as últimas seleções do usuário.
4.  **Backend (Para Dados Maiores ou Atualizações Dinâmicas):**
    *   Se o volume de dados se tornar muito grande, em vez de embutir todo o JSON no HTML, a página poderia fazer requisições a um backend Python (Flask, FastAPI) para obter apenas os dados do gráfico solicitado. Isso também permitiria atualizações de dados sem regenerar todo o HTML.
5.  **Autenticação e Dados de Múltiplos Usuários:**
    *   Para uma aplicação mais robusta, um sistema de login poderia ser implementado.
6.  **Exportação de Gráficos:**
    *   Adicionar funcionalidade para o usuário baixar os gráficos como imagens.
7.  **Testes e Refinamento Contínuo da UX/UI:**
    *   Coletar feedback e continuar aprimorando a interface.

Este projeto serve como uma base sólida para uma ferramenta de análise de treino pessoal poderosa e customizável.